In [ ]:
import pickle
import numpy as np
import random
from tensorflow.keras.layers import Input, Dense, Embedding, GRU
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 80000
embedding_dim = 100
max_len = 500


In [19]:
filePath_x = './X_train.txt'
filePath_y = './Y_train.txt'
filePath_valx = './X_valid.txt'
filePath_valy = './Y_valid.txt'

with open(filePath_valx, 'rb') as lf:
    X_valid = pickle.load(lf)

with open(filePath_valy, 'rb') as lf:
    Y_valid = pickle.load(lf)
    
with open(filePath_x, 'rb') as lf:
    X_train = pickle.load(lf)

with open(filePath_y, 'rb') as lf:
    Y_train = pickle.load(lf)

In [20]:
Y_train_s = Y_train
Y_valid_s = Y_valid

Y_train_s = np.array(Y_train_s)
Y_valid_s = np.array(Y_valid_s)

index_14 = []
for idx, y in enumerate(Y_train):
    if y[14] == 1:
        #print("Yes")
        index_14.append(idx)
#print(index_14)

index_9 = []
for idx, y in enumerate(Y_train):
    if y[9] == 1:
        #print("Yes")
        index_9.append(idx)
#print(index_9)

index_13 = []
for idx, y in enumerate(Y_valid):
    if y[13] == 1:
        #print("Yes")
        index_13.append(idx)
#print(index_13)

index_8 = []
for idx, y in enumerate(Y_valid):
    if y[8] == 1:
        #print("Yes")
        index_8.append(idx)
#print(index_8)

for idx in index_14:
    Y_train_s[idx][4] = 1
for idx in index_9:
    Y_train_s[idx][13] = 1
    
Y_train_s = np.delete(Y_train_s, [0,9,14], axis=1)

for idx in index_13:
    Y_valid_s[idx][3] = 1
for idx in index_8:
    Y_valid_s[idx][12] = 1
    
Y_valid_s = np.delete(Y_valid_s, [8,13], axis=1)

Y_valid = Y_valid_s
Y_train = Y_train_s

(11397, 12)
(880525, 12)


In [21]:
tmp = [[x,y] for x, y in zip(X_train, Y_train)]
random.shuffle(tmp)
X_train = [n[0] for n in tmp]
Y_train = [n[1] for n in tmp]

In [27]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(Y_valid.shape)
print(Y_train.shape)
print(X_train.shape)
print(X_valid.shape)

(11397, 12)
(880525, 12)
(880525, 260)
(11397, 260)


In [ ]:
X_test_r, X_valid_r, Y_test_r, Y_valid_r = train_test_split(X_valid, Y_valid, test_size = 0.5, random_state=1, shuffle=True)
category = 12

inputs = Input(shape=(max_len,))
inputs = Embedding(vocab_size, embedding_dim)(inputs)
inputs = GRU(max_len)(inputs)
output = Dense(category, activation='softmax')(inputs)
model = Model(inputs=inputs, outputs=output) 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=40, batch_size=128 ,validation_data=(X_valid_r, Y_valid_r))
results = model.evaluate(X_test_r, Y_test_r)
print(results)